<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Copy_of_igbobert4a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -c https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
!wget -c https://raw.githubusercontent.com/chiamaka249/lacuna_pos_ner/main/language_corpus/ibo/ibo.txt
!wget -c https://github.com/chiamaka249/IgboNER/blob/main/config.json

--2021-12-10 00:45:31--  https://github.com/IgnatiusEzeani/IGBONLP/raw/master/ig_monoling/text.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip [following]
--2021-12-10 00:45:31--  https://raw.githubusercontent.com/IgnatiusEzeani/IGBONLP/master/ig_monoling/text.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7604282 (7.3M) [application/zip]
Saving to: ‘text.zip’

text.zip            100%[===================>]   7.25M  --.-KB/s    in 0.06s   

2021-12-10 00:45:31 (114 MB/s) - ‘text.zip’ saved [7604282/7604282]

--2021-12-10 00:45:31-- 

In [2]:
import zipfile
import os


def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

unzip("text.zip")
!rm text.zip

In [3]:
#copies the file "ibo.txt" to into the folder "text"
import shutil
shutil.move('/content/ibo.txt', '/content/text')



'/content/text/ibo.txt'

In [4]:
# import os
#import shutil
dir_name = "/content/text"
text=""
for fname in os.listdir(dir_name):
  fname = os.path.join(dir_name, fname)
  with open(fname, "r", encoding="utf8") as datafile:
    text = text+"\n"+datafile.read()

with open("data.txt", "w", encoding="utf8") as datafile:
  datafile.write(text)

shutil.rmtree("text")

In [5]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x_83yala
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-x_83yala
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 9.0 MB/s 
     |████████████████████████████████| 895 kB 86.8 MB/s 
     |████████████████████████████████| 3.3 MB 73.7 MB/s 
     |████████████████████████████████| 61 kB 662 kB/s 
  Created wheel for transformers: filename=transformers-4.14.0.dev0-py3-none-any.whl size=3320667 sha256=16209dbd306b3a82a3bf60064b4f4031f080a4916e772f453e1e6c6af9c77c57
  Stored in directory: /tmp/pip-ephem-wheel-cache-thu00alc/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [6]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 19.2 s, sys: 1.24 s, total: 20.4 s
Wall time: 5.69 s


In [7]:
!mkdir igbo_bert4
tokenizer.save_model("igbo_bert4")

['igbo_bert4/vocab.json', 'igbo_bert4/merges.txt']

In [8]:
shutil.move('/content/config.json', '/content/igbo_bert4') 

'/content/igbo_bert4/config.json'

In [9]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert4/vocab.json",
    "./igbo_bert4/merges.txt",
)

In [10]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [11]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.").tokens

['<s>',
 'Simone',
 'Ġgara',
 'Ġá»¥ka',
 'Ġá»¥nyahá»¥',
 'Ġgu',
 'á»į',
 'Ġegwu',
 'Ġma',
 'Ġga',
 '-',
 'kwa',
 'Ġtaa',
 '.',
 '</s>']

In [12]:
# Check that we have a GPU
!nvidia-smi

Fri Dec 10 00:46:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [14]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [15]:
#from google.colab import files
#files.upload()

In [16]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert4", max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [17]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [18]:
model.num_parameters()
# => 83 million parameters

83504416

In [19]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/data.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 32.1 s, sys: 1.23 s, total: 33.4 s
Wall time: 15.2 s


In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert4",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [22]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 391448
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30585
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.383200
1000,5.338800
1500,4.878500
2000,4.552000
2500,4.310400
3000,4.143100
3500,3.994600
4000,3.892700
4500,3.741200
5000,3.673700


Saving model checkpoint to ./igbo_bert4/checkpoint-10000
Configuration saved in ./igbo_bert4/checkpoint-10000/config.json
Model weights saved in ./igbo_bert4/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-20000
Configuration saved in ./igbo_bert4/checkpoint-20000/config.json
Model weights saved in ./igbo_bert4/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./igbo_bert4/checkpoint-30000
Configuration saved in ./igbo_bert4/checkpoint-30000/config.json
Model weights saved in ./igbo_bert4/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [igbo_bert4/checkpoint-10000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 3h 4min, sys: 12min 37s, total: 3h 16min 37s
Wall time: 3h 15min 44s


TrainOutput(global_step=30585, training_loss=3.1495825427798874, metrics={'train_runtime': 11744.2923, 'train_samples_per_second': 166.655, 'train_steps_per_second': 2.604, 'total_flos': 3.365972073574656e+16, 'train_loss': 3.1495825427798874, 'epoch': 5.0})

In [23]:
trainer.save_model("./igbo_bert4")

Saving model checkpoint to ./igbo_bert4
Configuration saved in ./igbo_bert4/config.json
Model weights saved in ./igbo_bert4/pytorch_model.bin


In [24]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.14.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [25]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.036398716270923615,
  'sequence': 'Abụ m Maazị Ọkafọ.',
  'token': 5307,
  'token_str': ' Ọkafọ'},
 {'score': 0.01226269081234932,
  'sequence': 'Abụ m Maazị A.',
  'token': 348,
  'token_str': ' A'},
 {'score': 0.010232401080429554,
  'sequence': 'Abụ m Maazị O.',
  'token': 381,
  'token_str': ' O'},
 {'score': 0.009688876569271088,
  'sequence': 'Abụ m Maazị Chineke.',
  'token': 394,
  'token_str': ' Chineke'},
 {'score': 0.0048571135848760605,
  'sequence': 'Abụ m Maazị M.',
  'token': 448,
  'token_str': ' M'}]

In [26]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.2381240725517273,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.08440182358026505,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 724,
  'token_str': ' nwunye'},
 {'score': 0.07014906406402588,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.03455997258424759,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 623,
  'token_str': ' nwaanyị'},
 {'score': 0.024705424904823303,
  'sequence': 'Nwaanyị na ha ji na akara.',
  'token': 296,
  'token_str': ' ha'}]

In [27]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.2959575653076172,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 707,
  'token_str': ' ọjọọ'},
 {'score': 0.12293197959661484,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 427,
  'token_str': ' niile'},
 {'score': 0.07944665849208832,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.0650472342967987,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 496,
  'token_str': ' ọma'},
 {'score': 0.035585831850767136,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ike.',
  'token': 358,
  'token_str': ' ike'}]

In [28]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.07655657082796097,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị iche.',
  'token': 462,
  'token_str': ' iche'},
 {'score': 0.05872797220945358,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị mkpa.',
  'token': 607,
  'token_str': ' mkpa'},
 {'score': 0.050002582371234894,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị icheiche.',
  'token': 3738,
  'token_str': ' icheiche'},
 {'score': 0.03505287691950798,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị egwu.',
  'token': 705,
  'token_str': ' egwu'},
 {'score': 0.03378322347998619,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị nsọ.',
  'token': 715,
  'token_str': ' nsọ'}]

In [29]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12419361621141434,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.047471560537815094,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 870,
  'token_str': ' nri'},
 {'score': 0.024701394140720367,
  'sequence': 'Nwaanyị na eri ha na akara.',
  'token': 296,
  'token_str': ' ha'},
 {'score': 0.019818346947431564,
  'sequence': 'Nwaanyị na eri ego na akara.',
  'token': 591,
  'token_str': ' ego'},
 {'score': 0.01857335865497589,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 300,
  'token_str': ' ihe'}]

In [30]:
# The sun <mask>.
# =>

fill_mask("Gaanụ mee ndị <mask> niile ka ha bụrụ ndị na- eso ụzọ m  .") #= mba


[{'score': 0.36260899901390076,
  'sequence': 'Gaanụ mee ndị a niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.06406810879707336,
  'sequence': 'Gaanụ mee ndị mmadụ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 393,
  'token_str': ' mmadụ'},
 {'score': 0.06030040606856346,
  'sequence': 'Gaanụ mee ndị ọzọ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 434,
  'token_str': ' ọzọ'},
 {'score': 0.055031705647706985,
  'sequence': 'Gaanụ mee ndị ahụ niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 310,
  'token_str': ' ahụ'},
 {'score': 0.04922444000840187,
  'sequence': 'Gaanụ mee ndị m niile ka ha bụrụ ndị na- eso ụzọ m .',
  'token': 268,
  'token_str': ' m'}]

In [31]:
# The sun <mask>.
# =>

fill_mask("Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ <mask>.") #= Izrel


[{'score': 0.3182317912578583,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ya.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.21329301595687866,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ Izrel.',
  'token': 680,
  'token_str': ' Izrel'},
 {'score': 0.20545001327991486,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ mmadụ.',
  'token': 393,
  'token_str': ' mmadụ'},
 {'score': 0.04033781215548515,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ ha.',
  'token': 296,
  'token_str': ' ha'},
 {'score': 0.017210429534316063,
  'sequence': 'Jehova họpụtara Mozis ka ọ bụrụ onye ndú ụmụ anụmanụ.',
  'token': 1601,
  'token_str': ' anụmanụ'}]

In [32]:
# The sun <mask>.
# =>

fill_mask("Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka <mask> Haram.") #= Boko


[{'score': 0.8794072866439819,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Boko Haram.',
  'token': 2535,
  'token_str': ' Boko'},
 {'score': 0.0101553313434124,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Akwụkwọ Haram.',
  'token': 1099,
  'token_str': ' Akwụkwọ'},
 {'score': 0.009289715439081192,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Super Haram.',
  'token': 3199,
  'token_str': ' Super'},
 {'score': 0.005499729886651039,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Manchester Haram.',
  'token': 3278,
  'token_str': ' Manchester'},
 {'score': 0.003374180756509304,
  'sequence': 'Ụmụakwụkwọ Chibok anọọla ụbọchị 2000 n’ aka Man Haram.',
  'token': 2024,
  'token_str': ' Man'}]

In [33]:
# The sun <mask>.
# =>

fill_mask("Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado <mask> ọhụrụ a.") #= iwu


[{'score': 0.12909017503261566,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọrụ ọhụrụ a.',
  'token': 477,
  'token_str': ' ọrụ'},
 {'score': 0.05322573706507683,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ọchịchị ọhụrụ a.',
  'token': 719,
  'token_str': ' ọchịchị'},
 {'score': 0.036072392016649246,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ụlọọrụ ọhụrụ a.',
  'token': 1411,
  'token_str': ' ụlọọrụ'},
 {'score': 0.025011051446199417,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado ụka ọhụrụ a.',
  'token': 1154,
  'token_str': ' ụka'},
 {'score': 0.023319613188505173,
  'sequence': 'Nwunye Gọvanọ Ekiti steeti bụ Bisi Fayemi so na ndị na- akwado obodo ọhụrụ a.',
  'token': 576,
  'token_str': ' obodo'}]

In [34]:
# The sun <mask>.
# =>

fill_mask(" <mask> sị ka ehiwe ụlọikpe pụrụiche maka mpụ.") #= Buhari


[{'score': 0.5028418898582458,
  'sequence': 'A sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 37,
  'token_str': 'A'},
 {'score': 0.20332291722297668,
  'sequence': 'Ọ sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 336,
  'token_str': 'Ọ'},
 {'score': 0.03303634002804756,
  'sequence': 'Ha sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 513,
  'token_str': 'Ha'},
 {'score': 0.01529951673001051,
  'sequence': 'Igbo sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 3656,
  'token_str': 'Igbo'},
 {'score': 0.012158594094216824,
  'sequence': 'Ị sị ka ehiwe ụlọikpe pụrụiche maka mpụ.',
  'token': 488,
  'token_str': 'Ị'}]

In [35]:
# The sun <mask>.
# =>

fill_mask("Ala <mask>  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.") #= Naịjirịa


[{'score': 0.1200433224439621,
  'sequence': 'Ala ahụ  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 310,
  'token_str': ' ahụ'},
 {'score': 0.0745198205113411,
  'sequence': 'Ala a  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.02661355398595333,
  'sequence': 'Ala Nigeria  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 1570,
  'token_str': ' Nigeria'},
 {'score': 0.0216804351657629,
  'sequence': 'Ala niile  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 427,
  'token_str': ' niile'},
 {'score': 0.02018386870622635,
  'sequence': 'Ala Igbo  ga- eweta ezi ọnọdụ nchekwa maka ndị chọrọ ịwebata ego n’ ọrụ ugbo.',
  'token': 900,
  'token_str': ' Igbo'}]

In [36]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.08720937371253967,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.00832695048302412,
  'sequence': 'Ọ bụ Nnamdia ka a na- arịa.',
  'token': 3283,
  'token_str': ' Nnamdi'},
 {'score': 0.00810943078249693,
  'sequence': 'Ọ bụ nwaa ka a na- arịa.',
  'token': 419,
  'token_str': ' nwa'},
 {'score': 0.005891024600714445,
  'sequence': 'Ọ bụ nwunyea ka a na- arịa.',
  'token': 724,
  'token_str': ' nwunye'},
 {'score': 0.004374524112790823,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1459,
  'token_str': ' Nwanna'}]

In [37]:
import shutil

In [38]:
shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [40]:
model_save_name = '/igbo_bert4.zip'
path = F"/content/gdrive/My Drive/igbo_bert/{model_save_name}"
torch.save(model.state_dict(), path)

In [41]:
#from google.colab import files
#files.download("/content/igbo_bert4.zip")

In [ ]:
gdrive/

In [42]:
shutil.move('/content/igbo_bert4','/content/gdrive/MyDrive/igbo_bert')

'/content/gdrive/MyDrive/igbo_bert/igbo_bert4'

In [43]:
shutil.copy('/content/gdrive/MyDrive/igbo_bert/igbo_bert4','/content/sample_data')

IsADirectoryError: ignored